In [1]:
import pandas as pd
import numpy as np
from some_functions import get_db

In [2]:
readers = pd.read_csv(r"C:\Users\a814811\OneDrive - Atos\RecommenderSystem\readers.csv")
readers = readers.rename(columns={"id":"user_id", "art_id":"nzz_id"})
art_db = get_db(r'C:\Users\a814811\OneDrive - Atos\RecommenderSystem\art_clean_wt_all_popularity.csv')
art_db = art_db.loc[:,['nzz_id','author','department','popularity']] #skrócenie do potrzebnych rzeczy

In [3]:
read_counts = readers["user_id"].value_counts(sort=True)
read_counts = read_counts.rename_axis("user_id").reset_index(name="read_count")

# Biorę pod uwagę tylko użytkowników, którzy przeczytali minimum 5 artykułów
min_read_count = 3
read_counts = read_counts[read_counts["read_count"] > min_read_count]

readers = readers[readers["user_id"].isin(read_counts["user_id"])]

In [4]:
# Train/Test split
from sklearn.model_selection import train_test_split

random_state = None
readers_train, readers_test = train_test_split(readers,
                                   stratify=readers["user_id"], 
                                   test_size=0.20,
                                   random_state=random_state)
                                   
print(f"Train set size {len(readers_train)}")
print(f"test set size {len(readers_test)}")
unique_train = readers_train["nzz_id"].unique()
unique_test = readers_test["nzz_id"].unique()
print(f"n users in test {len(unique_test)}")
print(f"n users in train {len(unique_train)}")

Train set size 22284
test set size 5571
n users in test 4023
n users in train 10005


In [5]:

from p2 import *
from model_evaluator import ModelEvaluator
model_evaluator = ModelEvaluator(k_list = [5, 10, 15])

### sprawdzam submodele i główny
#### dla authora są mocno zaniżone wyniki, bo często nie rekomenduje nic a test tego nie uwzględnia, podobnie dla department (w mniejszym stopniu)
#### merged i popularity działają normalnie (zawsze coś jest rekomednowane)

In [6]:
# popularity model
p_model = Popularity_model(art_db,readers)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(p_model, readers, readers_train, readers_test)
print('\nGlobal metrics:\n%s' % cf_global_metrics)

999 users processed

Global metrics:
{'modelName': 'popularity', 'recall@5': 0.23830793650793652, 'precision@5': 0.0476615873015873, 'f1_score@5': 0.07943597883597883, 'ndcg@5': 0.16635031631663777, 'personalization@5': 0.9921175842509176, 'recall@10': 0.35951785714285717, 'precision@10': 0.03595178571428572, 'f1_score@10': 0.06536688311688311, 'ndcg@10': 0.2044001333208505, 'personalization@10': 0.9921175842509176, 'recall@15': 0.44040238095238093, 'precision@15': 0.02936015873015873, 'f1_score@15': 0.055050297619047624, 'ndcg@15': 0.2269823200521949, 'personalization@15': 0.9921175842509176}


In [6]:
# author popularity model
p_model = Popularity_model_author(art_db,readers)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(p_model, readers, readers_train, readers_test)
print(f'\nGlobal metrics:\n{cf_global_metrics}')

999 users processed
c:\Users\a814811\OneDrive - Atos\RecommenderSystem\Piotr\model\Kamila_test_model\model_evaluator.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predicted = np.array(predicted)


TypeError: unhashable type: 'list'

In [6]:
# department popularity model
p_model = Popularity_model_department(art_db,readers)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(p_model, readers, readers_train, readers_test)
print(f'\nGlobal metrics:\n{cf_global_metrics}')

999 users processed
c:\Users\a814811\OneDrive - Atos\RecommenderSystem\Piotr\model\Kamila_test_model\model_evaluator.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predicted = np.array(predicted)


TypeError: unhashable type: 'list'

In [7]:
# merged model
p_model = Popularity_model_final(art_db,readers)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(p_model, readers, readers_train, readers_test)
print(f'\nGlobal metrics:\n{cf_global_metrics}')

999 users processed

Global metrics:
{'modelName': 'final', 'recall@5': 0.23528253968253968, 'precision@5': 0.04705650793650794, 'f1_score@5': 0.07842751322751323, 'ndcg@5': 0.1613260817774572, 'personalization@5': 0.9920028028028028, 'recall@10': 0.3577218253968254, 'precision@10': 0.03577218253968255, 'f1_score@10': 0.06504033189033188, 'ndcg@10': 0.2000917699602541, 'personalization@10': 0.9920028028028028, 'recall@15': 0.44594404761904766, 'precision@15': 0.029729603174603172, 'f1_score@15': 0.05574300595238096, 'ndcg@15': 0.22559260989706184, 'personalization@15': 0.9920028028028028}


In [20]:
cf_detailed_results_df['model'] = ['merged']*1000
cf_detailed_results_df.to_csv("test_results2", encoding="utf-8", index=False)

## Wizualizacja

In [15]:
from matplotlib import pyplot as plt